Guides :

- https://openpyxl.readthedocs.io/en/stable/
- https://automatetheboringstuff.com/chapter12/

# Import des librairies

In [ ]:
from pathlib import Path

import openpyxl
import pandas as pd

# Variables globales

In [ ]:
data = Path("../data")  # Chemin vers le dossier data


# Exploration du fichier excel des modules

## Variables globales

In [ ]:
colonnes = [
    'Identifiant',
    'Abréviation',
    'Code PPN',
    'Nom complet',
    'Promotion',
    'Période',
    'Responsable'
]

In [ ]:
module_wb = openpyxl.load_workbook(
    data / "liste_modules.xlsx"
)  # Objet python représentant le fichier excel
print(
    module_wb.sheetnames
)  # On récupère le nom des feuilles du fichier excel, ici il n'y en a qu'une seule
module_sheet = module_wb[
    module_wb.sheetnames[0]
]  # Objet python représentant la première feuille du fichier excel


# Obtention des indices de début pour chaque filière

In [ ]:
currentPathway = module_sheet[f'C{1}'].value

pathways = {currentPathway:[1]}

for i in range(1, module_sheet.max_row+1):
    if module_sheet[f'C{i}'].value != currentPathway:
        pathways[currentPathway].append(i-1)
        currentPathway = module_sheet[f'C{i}'].value
        pathways[currentPathway] = [i]

pathways[currentPathway].append(module_sheet.max_row)

print(pathways)

# Obtention des modules d'informatique

In [ ]:
selectedPathway = 'INFO'

firstRowCS = pathways[selectedPathway][0]
lastColumnCoordinates = f'{openpyxl.utils.cell.get_column_letter(module_sheet.max_column)}{firstRowCS}'
firstLine = module_sheet[f'A{firstRowCS}':lastColumnCoordinates]
for rowOfCellObjects in firstLine:
    i = 0
    for cellObj in rowOfCellObjects:
        print(i, cellObj.coordinate, cellObj.value)
        i += 1

## Mise en place de la structure de donnée interne

In [ ]:
demonstration_df = pd.DataFrame(columns=colonnes)
demonstration_df.head()

### Informations directement disponibles

In [ ]:
# Depuis les données brutes on peut remplir les 2 premières colonnes
rawData = {
    'Identifiant': firstLine[0][0].value,
    'Abréviation' : firstLine[0][1].value
}

currentEntry = pd.DataFrame(rawData, index=[0]) # On créé un dataframe contenant les données bruts

demonstration_df = pd.concat([demonstration_df, currentEntry]).reset_index(drop=True) # On l'ajoute au dataframe principal

demonstration_df

In [ ]:
# Dans notre cas des informations sont redondantes

demonstration_df['Code PPN'] = demonstration_df['Identifiant']
demonstration_df['Nom complet'] = demonstration_df['Abréviation']

demonstration_df

### Informations à déduire

In [ ]:
description = firstLine[0][11].value

print(description)

descriptionSegmented = description.replace('/', ',').split(',')

print(descriptionSegmented)

# Removing
indexToRemove = []

for i in range(len(descriptionSegmented)):
    processedString = descriptionSegmented[i].strip()

    if processedString == '':
        indexToRemove.append(i)
    else:
        descriptionSegmented[i] = processedString

for index in reversed(indexToRemove):
    descriptionSegmented.pop(index)

print(descriptionSegmented)

In [ ]:
semesterSegment = descriptionSegmented[1]
print(semesterSegment.strip('Semestre').strip())
semesterNumber = semesterSegment.strip('Semestre').strip()[1]

currentEntry['Période'] = f'S{semesterNumber}'

year = int(semesterNumber)%2 # 2 semestres par année

yearGroup = f'{selectedPathway}{year}'
print(yearGroup)

currentEntry['Promotion'] = yearGroup

demonstration_df = pd.concat([demonstration_df, currentEntry]).reset_index(drop=True) # On l'ajoute au dataframe principal

demonstration_df

Pour le responsable on s'attend à son identifiant, il faut implémenter une fonction permettant de récupérer son identifiant; on ne peut pas juste prendre ses initiales (cas où 2 individus ont les mêmes initiales)

In [ ]:
identity = firstLine[0][4].value
print(identity)

#TODO: recherche d'identifiant

initials = ''.join([string[0].upper() for string in identity.split(' ')])

print(initials)

## Regroupement des procédures d'extraction d'informations

In [ ]:
def extractModuleInformation(moduleExcelLine: tuple):
    # Return a single row DataFrame containing information regarding a specific module

    moduleLine = moduleExcelLine[0]
    moduleDataFrame = pd.DataFrame(columns=colonnes)

    # Identfiant, Abréviation, Code PPN, Nom complet

    rawData = {
        'Identifiant': moduleLine[0].value,
        'Abréviation' : moduleLine[1].value
    }

    rawDataFrame = pd.DataFrame(rawData, index=[0]) # On créé un dataframe contenant les données bruts

    moduleDataFrame = pd.concat([moduleDataFrame, rawDataFrame]).reset_index(drop=True) # On l'ajoute au dataframe principal

    moduleDataFrame["Code PPN"] = moduleDataFrame["Identifiant"]
    moduleDataFrame["Nom complet"] = moduleDataFrame["Abréviation"]

    # Période et Promotion

    description = moduleLine[11].value

    #print(description)

    descriptionSegmented = description.replace("/", ",").split(",")

    #print(descriptionSegmented)

    # Removing trailing spaces and empty strings
    indexToRemove = []

    for i in range(len(descriptionSegmented)):
        processedString = descriptionSegmented[i].strip()

        if processedString == "":
            indexToRemove.append(i)
        else:
            descriptionSegmented[i] = processedString

    for index in reversed(indexToRemove):
        descriptionSegmented.pop(index)

    semesterSegment = descriptionSegmented[1]
    #print(semesterSegment.strip("Semestre").strip())
    semesterNumber = semesterSegment.strip("Semestre").strip()[1]

    moduleDataFrame["Période"] = f"S{semesterNumber}"

    year = int(semesterNumber) // 2 + 1  # 2 semestres par année

    yearGroup = f"{selectedPathway}{year}"
    #print(yearGroup)

    moduleDataFrame["Promotion"] = yearGroup

    # Responsable

    # TODO: avec le nom et prénom du responsable, il faut pouvoir obtenir son identifiant qui peut ne pas être ses initiales

    tutorIdentity = moduleLine[4].value
    tutorInitials = ''.join([string[0].upper() for string in tutorIdentity.split(' ')])
    moduleDataFrame["Responsable"] = tutorInitials

    return moduleDataFrame


In [ ]:
extractModuleInformation(firstLine).head()

In [ ]:
def batchModuleInformation(excelSheet, rowRange):
    modulesDF = pd.DataFrame(columns=colonnes)

    lastColumnLetter = f'{openpyxl.utils.cell.get_column_letter(excelSheet.max_column)}'

    for rowNumber in range(rowRange[0], rowRange[1]):
        lastColumnCoordinates = f'{lastColumnLetter}{rowNumber}'
        processedLine = excelSheet[f'A{rowNumber}':lastColumnCoordinates]
        processedDataFrame = extractModuleInformation(processedLine)

        modulesDF = pd.concat([modulesDF, processedDataFrame]).reset_index(drop=True) 

    return modulesDF

batchModuleInformation(module_sheet, pathways['INFO']).tail()

In [ ]:
def DataFramesToExcel(dataframes:[pd.DataFrame], sheetNames:[str], output:str or Path):
    # Create an Excel file in which each sheet is associated with a dataframe in the list "dataframes" and its name is indicated in the list "sheetNames"

    with pd.ExcelWriter(output) as writer:
        for index in range(len(sheetNames)):
            dataframes[index].to_excel(writer, sheet_name=sheetNames[index], index=False)

moduleDataFrame = batchModuleInformation(module_sheet, pathways['INFO'])

DataFramesToExcel([moduleDataFrame], ['Modules'], Path('../output/database_file.xlsx'))